In [1]:
!ls

homeriskactivity.csv test.ipynb           train.txt


In [1]:
import pandas as pd
df = pd.read_csv('./homeriskactivity.csv')

/var/folders/4c/pwmkzqlj7h56788tbtv1t5tr0000gn/T/ipykernel_12405/336853481.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./homeriskactivity.csv')


In [5]:
import csv
import sys
import re
pattern = "https?://[\w/:%#\$&\?\(\)~\.=\+\-]+"
for index, row in df.iterrows():
    if not re.match(pattern, row[0]) or not re.match(pattern, row[1]) or not re.match(pattern, row[2]):
        continue
    with open("./train_2.txt", "a") as f:
        f.write("\""+row[0]+"\"," + "\""+row[1]+"\",""\""+row[2]+"\"\n")

KeyboardInterrupt: 

In [6]:
%cd ../

/Users/hammik/Documents/KnowledgeGraph/dgl-ke/python/my_task


In [7]:
!DGLBACKEND=pytorch dglke_train --model_name TransE_l2  --data_path homeriskactivity_2 --dataset homeriskactivity_2 --format raw_udd_hrt --delimiter "," --data_files train.txt \
--batch_size 2 --neg_sample_size 2 --hidden_dim 4 --gamma 19.9 --lr 0.25 --max_step 10 --log_interval 2 \
-adv --regularization_coef 1.00E-09 

Reading train triples....
Finished. Read 1552261 train triples.
/usr/local/lib/python3.10/site-packages/dgl/_deprecate/graph.py:1023: DGLWarning: multigraph will be deprecated.DGL will treat all graphs as multigraph in the future.
  dgl_warning("multigraph will be deprecated." \
|Train|: 1552261
Total initialize time 6.190 seconds
[proc 0][Train](2/10) average pos_loss: 0.0003131340636173263
[proc 0][Train](2/10) average neg_loss: 10.591350078582764
[proc 0][Train](2/10) average loss: 5.2958314418792725
[proc 0][Train](2/10) average regularization: 1.6827455624479626e-06
[proc 0][Train] 2 steps take 0.468 seconds
[proc 0]sample: 0.248, forward: 0.064, backward: 0.154, update: 0.002
[proc 0][Train](4/10) average pos_loss: 0.1972399428486824
[proc 0][Train](4/10) average neg_loss: 10.194532871246338
[proc 0][Train](4/10) average loss: 5.1958863735198975
[proc 0][Train](4/10) average regularization: 2.1031935375503963e-06
[proc 0][Train] 2 steps take 0.006 seconds
[proc 0]sample: 0.001, f

In [11]:
import pandas
entities = pandas.read_csv('./homeriskactivity_2/entities.tsv', delimiter=',', header=None)

In [12]:
from rdflib import *
from SPARQLWrapper import SPARQLWrapper, JSON

activities = []
queryString = """
PREFIX ex: <http://example.org/virtualhome2kg/instance/>
PREFIX : <http://example.org/virtualhome2kg/ontology/>
select DISTINCT * where {
    ?activity :virtualHome ex:scene1 .
} 
"""
endpointURI = "http://kgrc4si.ml:7200/repositories/KGRC4SIv0"
sparql = SPARQLWrapper(endpointURI)
sparql.setQuery(queryString)
sparql.setReturnFormat(JSON)

json = sparql.query().convert()
bindings = json['results']['bindings']
results = bindings
for entity in results:
  activities.append(entity["activity"]["value"])

for a in activities:
    print("activity: ", a)
    print(entities.loc[entities[1] == a,0])
    with open("./head.list", "a") as f:
        f.write(str(entities.loc[entities[1] == a][0].tolist()[0])+"\n")

activity:  http://example.org/virtualhome2kg/instance/admire_paintings_scene1
193713    193713
Name: 0, dtype: int64
activity:  http://example.org/virtualhome2kg/instance/browse_internet_scene1
190016    190016
Name: 0, dtype: int64
activity:  http://example.org/virtualhome2kg/instance/brush_teeth_scene1
186867    186867
Name: 0, dtype: int64
activity:  http://example.org/virtualhome2kg/instance/carry_box_scene1
183167    183167
Name: 0, dtype: int64
activity:  http://example.org/virtualhome2kg/instance/clean_desk_scene1
110880    110880
Name: 0, dtype: int64
activity:  http://example.org/virtualhome2kg/instance/drink_scene1
98795    98795
Name: 0, dtype: int64
activity:  http://example.org/virtualhome2kg/instance/find_some_food_scene1
91858    91858
Name: 0, dtype: int64
activity:  http://example.org/virtualhome2kg/instance/go_to_sleep_scene1
88148    88148
Name: 0, dtype: int64
activity:  http://example.org/virtualhome2kg/instance/go_to_toilet_scene1
83813    83813
Name: 0, dtype: in

In [13]:
!DGLBACKEND=pytorch dglke_predict --model_path ckpts/TransE_l2_homeriskactivity_2_0/ --format 'h_r_*' --data_files homeriskactivity_2/head.list homeriskactivity_2/relation.list --score_func logsigmoid --topK 5 --exec_mode 'batch_head'

ckpts/TransE_l2_homeriskactivity_2_0/config.json
{'model_name': 'TransE_l2', 'data_path': 'homeriskactivity_2', 'dataset': 'homeriskactivity_2', 'format': 'raw_udd_hrt', 'data_files': ['train.txt'], 'delimiter': ',', 'save_path': 'ckpts/TransE_l2_homeriskactivity_2_0', 'no_save_emb': False, 'max_step': 10, 'batch_size': 2, 'batch_size_eval': 8, 'neg_sample_size': 2, 'neg_deg_sample': False, 'neg_deg_sample_eval': False, 'neg_sample_size_eval': 197528, 'eval_percent': 1, 'no_eval_filter': False, 'log_interval': 2, 'eval_interval': 10000, 'test': False, 'num_proc': 1, 'num_thread': 1, 'force_sync_interval': -1, 'hidden_dim': 4, 'lr': 0.25, 'gamma': 19.9, 'double_ent': False, 'double_rel': False, 'neg_adversarial_sampling': True, 'adversarial_temperature': 1.0, 'regularization_coef': 1e-09, 'regularization_norm': 3, 'pairwise': False, 'loss_genre': 'Logsigmoid', 'margin': 1.0, 'gpu': [-1], 'mix_cpu_gpu': False, 'valid': False, 'rel_part': False, 'async_update': False, 'has_edge_importance

In [16]:
result = pandas.read_csv('./result.tsv', delimiter='\t')

In [17]:
relations = pandas.read_csv('./homeriskactivity_2/relations.tsv', delimiter=',', header=None)
import csv
for index, row in result.iterrows():   
    #a = entities.loc[entities[0] == row['head'],:]
    head = entities.loc[entities[0] == row['head']][1].tolist()[0]
    rel = relations.loc[relations[0] == row['rel']][1].tolist()[0]
    tail = entities.loc[entities[0] == row['tail']][1].tolist()[0]

    if "event" in tail:
        with open("./riskFactor_predict_link.csv", "a") as f:
            writer = csv.writer(f)
            writer.writerow([head, rel, tail])

In [18]:
!DGLBACKEND=pytorch dglke_emb_sim --emb_file ckpts/TransE_l2_homeriskactivity_2_0/homeriskactivity_2_TransE_l2_entity.npy --format 'l_*' --data_files homeriskactivity_2/head_lisk.list  --topK 10000

Inference Done
The result is saved in result.tsv


In [19]:
result = pandas.read_csv('./result.tsv', delimiter='\t')

In [20]:
import csv
for index, row in result.iterrows():   
    #a = entities.loc[entities[0] == row['head'],:]
    left = entities.loc[entities[0] == row['left']][1].tolist()[0]
    right = entities.loc[entities[0] == row['right']][1].tolist()[0]
    score = row['score']
    if "event" in right:
        with open("./riskFactor_5.csv", "a") as f:
            writer = csv.writer(f)
            writer.writerow([left, right, score])